## Coastal Cookbook: How to synchronize SCHISM outputs to work with T-Route if there are different datums

This notebook will walk through how to synchronize SCHISM output files based on the coastal mesh datum and the t-route datum. 

##### Required files:
- Crosswalk points for the v2.2 hydrofabric in the AtlGulfCrosswalk domain 
  - obtained by running the `datum-sync/tools/create_coastal_csv.py` using the `conus_nextgen.gpkg` file with the latest hydrofabric flowlines and the polygons denoting the crosswalk points from the `ngen-forcing/coastal/Coastal_Modeling_Preprocessing_Scripts/Coastal_Data/Semi_Circle_Polygons/Atlantic/AtlGulf/AtlGulf_InflowsOutflows_CCBuff.shp` file
- Outputs from the SCHISM coastal model
  - Using the outputs from an NWM coastal test case in the OE at `/efs/Zhengtao.Cui/schism_forecast_test/schism_forecast_2025051100/outputs/`

In [ ]:
import os
import re
from pathlib import Path

import geopandas as gpd
import numpy as np
import xarray as xr
import pandas as pd
from pyprojroot import here
from scipy.spatial import cKDTree

project_root = here()
schism_output_dir = Path("/efs/Zhengtao.Cui/schism_forecast_test/schism_forecast_2025051100/outputs/")

First, we're going to open all of the schism outputs into one xarray dataset to view the contents

In [ ]:
outputfiles = sorted(
    schism_output_dir.glob('out2d_*.nc'), 
    key=lambda x: float(re.findall(r"out2d_(\d+)\.nc", x.name)[0])
)

In [ ]:
ds = xr.open_mfdataset(outputfiles, engine="netcdf4")

In [ ]:
ds

Next, We're going to make a kd-tree for nearest neighbor searches within the mesh. Through this, we can determine where the source points we care about are located and their indices

In [ ]:
schism_nodes = np.column_stack([ds.isel(time=0).SCHISM_hgrid_node_x.values,ds.isel(time=0).SCHISM_hgrid_node_y.values])
tree = cKDTree(schism_nodes)

Now that we have the kd-tree, we're going to determine which crosswalk points from the AtlGulf pre-computed file and subset our schism outputs

In [ ]:
cw = gpd.read_file(project_root / "AtlGulf_hf_cross_walk.gpkg")
cw_df = pd.read_csv(project_root / "AtlGulf_hf_cross_walk.csv")
fp = gpd.read_file(project_root / "troute_run_dir/domain/lower_colorado.gpkg", layer="flowpaths")
intersection_points = cw_df[cw_df["id"].isin(fp["id"])]
cw = cw[cw["id"].isin(fp["id"])]
coords = intersection_points[["longitude", "latitude"]].values
distances, indices = tree.query(coords)
schism_subset = ds.isel(nSCHISM_hgrid_node=indices)

Once the SCHISM data is subset, we can run our BMI-module for datum synchronization

#### NOTE: for this example we are pretending the SCHISM OUTPUT uses EPSG:4979. This is not true as SCHISM uses NAVD88. This is an example just to show how the projection could be done if the T-Route and SCHISM datums were different

In [ ]:
#------------------------------------------------------
#Initialize the BMI model
#------------------------------------------------------
os.chdir(project_root / "extern/datum-sync")
import yaml
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr

from datum_sync.bmi.bmi_datum_sync import BmiDatumSync

data = {
    "crs_input": 4979,  #WGS84 + Height  
    "crs_output": 5498,  #NAVD83 + NAVD88
    "z_warn": True,
}
config_file = project_root / "source_sync.yaml"
with open(config_file, 'w') as file:
    yaml.dump(data, file)

model = BmiDatumSync()
model.initialize(config_file)
#------------------------------------------------------
#Process SCHISM dataset time series
#------------------------------------------------------

# Extract coordinates (constant across time)
lon_values = schism_subset.SCHISM_hgrid_node_x.isel(time=0).values
lat_values = schism_subset.SCHISM_hgrid_node_y.isel(time=0).values

n_times = schism_subset.time.shape[0]
n_points = len(lon_values)

# Initialize output arrays
converted_coords = np.zeros((n_times, n_points, 3))  # [time, points, [lon,lat,elev]]
print(f"Converting {n_points} nodes across {n_times} time steps...")

# Process each time step
for t in range(n_times):
    # Get elevation values for this time step
    elev_values = schism_subset.elevation.isel(time=t).values
    
    # Set BMI input values
    model.set_value("longitude", lon_values.tolist())
    model.set_value("latitude", lat_values.tolist()) 
    model.set_value("elevation", elev_values.tolist())
    
    # Run conversion
    model.update()
    
    # Get converted coordinates
    dest_array = np.zeros(n_points * 3)
    model.get_value("coordinates__output", dest_array)
    converted_coords[t, :, :] = dest_array.reshape((n_points, 3), order="F")

# ------------------------------------------------------
# Update SCHISM dataset with converted values
# ------------------------------------------------------

schism_converted = schism_subset.copy(deep=True)

# Update coordinates
schism_converted['SCHISM_hgrid_node_x'] = (
    ('time', 'nSCHISM_hgrid_node'), 
    converted_coords[:, :, 0]
)
schism_converted['SCHISM_hgrid_node_y'] = (
    ('time', 'nSCHISM_hgrid_node'), 
    converted_coords[:, :, 1]
)
schism_converted['elevation'] = (
    ('time', 'nSCHISM_hgrid_node'),
    converted_coords[:, :, 2]
)

# Convert Depth
schism_converted['depth'] = np.abs(schism_subset.depth)

# ------------------------------------------------------
# Save results
# ------------------------------------------------------
keep_vars = ['elevation', 'SCHISM_hgrid_node_x', 'SCHISM_hgrid_node_y', 'depth']
schism_minimal = schism_converted[keep_vars]

# Optional Saving of results
# output_path = project_root / "datum_sync_schism_output.nc"
# schism_minimal.to_netcdf(output_path)

print("Finalize")
model.finalize()

Now that the datum is "synchronized", we can plot the outputs using geopandas to verify where our SCHISM outputs can be used by the diffusive wave model. 

#### NOTE: the outputs of this notebook are based on the Specification from the t-route wiki for coastal modeling

In [ ]:
from shapely.geometry import Point

lon, lat = schism_minimal.SCHISM_hgrid_node_x.isel(time=0).values, schism_minimal.SCHISM_hgrid_node_y.isel(time=0).values
geometry = [Point(x, y) for x, y in zip(lon, lat)]
gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')
m = fp.explore(color="blue")
gdf.explore(m=m, color="red")

If you want to view the values of the schism dataset before and after the datum sync:

In [ ]:
# Before the datum change at the output timestep
schism_subset.elevation.isel(time=-1).values

In [ ]:
# After the datum change at the output timestep
schism_minimal.elevation.isel(time=-1).values

In conclusion: 
- This work demonstrates the integration of the coastal and inland datums through the use of a BMI module to synchronize datums
- This work uses SCHISM outputs and formats them to fit the T-Route diffusive wave specification
- This work provides data which can be inspected for a "sample test case" 